In [ ]:
#################################################################
#                                                               #
#    Author: Alan Bauer, alan.bauer@earlham.ac.uk               #
#    Date: 2018.11.15, V0.1                                     #
#    Version: 0.1 on Bitbucket                                  #
#                                                               #
#################################################################

# Attempting to segment aerial wheat images into individual plots
# and extract data from each plot.

In [1]:
# Imports
import numpy as np
import cv2

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import model_from_json
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import sgd
from keras import backend as K
from keras import utils as np_utils
import keras

import os
import glob
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [2]:
# Process into 9x9 images. It assumes that if they aren't 9x9 they are 11x11
# because that is the size I first started classifying. In that case the
# outermost pixels are removed so the center pixel remains the same.
def preprocess(img):
    h,w = img.shape[:2]
    if h is 9 and w is 9:
        return img
    
    up_img = img[1:10,1:10]

    return up_img

In [3]:
# Just testing that the preprocess function properly resizes images
img = "datasets/soil/soil/0_0_04_28.png"
img = cv2.imread(img)
up_img = preprocess(img)
cv2.imwrite("test.png",up_img)
print(up_img.shape)

(9, 9, 3)


In [4]:
# Load images from a base directory that contains two subfolders,
# one for each of the two classes
def load_imgs(base_path):
    pos_class = "soil"
    neg_class = "not_soil"
    
    files = []
    ext = ".png" # May need to make this more comprehensive on a different OS
    
    data = []
    labels = []
    
    for (top_dir, dirs, filenames) in os.walk(base_path):
        for filename in filenames:
            if filename.endswith(ext):
                f = os.path.join(top_dir,filename)
                img = cv2.imread(f)
                img = preprocess(img)
                data.append(img)

                label = f.split(os.path.sep)[-2]
                labels.append(label)
    
    return (np.array(data), np.array(labels))

In [5]:
# Build the CNN
def build_model(width, height, depth, classes):
    model = Sequential()
    shape = (height, width, depth)
    
    if K.image_data_format() == "channels_first":
        shape = (depth, height, width)
    
    model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=shape))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(classes,activation="softmax"))
    
    return model

In [6]:
# Load the data and split it into training and testing sets
data_dir = "datasets/soil/"
(data, labels) = load_imgs(data_dir)
data = data.astype("float") / 255.0

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=1)

trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

trainY = np_utils.to_categorical(trainY)
testY = np_utils.to_categorical(testY)

print(len(data))


3664


In [7]:
# Build the model
model = build_model(9,9,3,2)
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [8]:
# Train the model, if you are loading a model from disk, skip this step
print("[INFO] training network")

H = model.fit(trainX, trainY, validation_data=(testX,testY),batch_size=32, epochs=100, verbose=1)

print("[INFO] evaluating network")
predictions = model.predict(testX, batch_size=32,verbose=1)
target_names = ["not soil","soil"]
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1),target_names=target_names))


[INFO] training network
Train on 2748 samples, validate on 916 samples
Epoch 1/100
2748/2748 [==============================] - 12s 4ms/step - loss: 0.2922 - acc: 0.9079 - val_loss: 0.3342 - val_acc: 0.8897
Epoch 2/100
2748/2748 [==============================] - 10s 3ms/step - loss: 0.1960 - acc: 0.9385 - val_loss: 0.3746 - val_acc: 0.8941
Epoch 3/100
2748/2748 [==============================] - 9s 3ms/step - loss: 0.1548 - acc: 0.9487 - val_loss: 0.2915 - val_acc: 0.9159
Epoch 4/100
2748/2748 [==============================] - 9s 3ms/step - loss: 0.1249 - acc: 0.9592 - val_loss: 0.0752 - val_acc: 0.9760
Epoch 5/100
2748/2748 [==============================] - 8s 3ms/step - loss: 0.1218 - acc: 0.9556 - val_loss: 0.1930 - val_acc: 0.9531
Epoch 6/100
2748/2748 [==============================] - 9s 3ms/step - loss: 0.1133 - acc: 0.9643 - val_loss: 0.1179 - val_acc: 0.9651
Epoch 7/100
2748/2748 [==============================] - 9s 3ms/step - loss: 0.0904 - acc: 0.9709 - val_loss: 0.1751 

KeyboardInterrupt: 

In [7]:
# Save or load model

#model.save("models/soil/model_5.h5")
model = load_model("models/soil/model_5.h5")

In [9]:
# Info about model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 9, 9, 32)          896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 32)          128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
batch_normalization_3 (Batch (None, 9, 9, 128)         512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
__________

In [10]:
# This extracts 9x9 images from a larger image and returns
# 4 numpy arrays, the images, an empty array for labels to be added
# and the coordinates of the top left corner of the images
def get_small_imgs_from_mosaic(img):
    h,w = img.shape[:2]
    step = 6
    size = 9
        
    step_x = 5
    step_y = 5
    
    images_to_pred = []
    X = []
    Y = []
    
    
    for x in range(0,w-size,step):
        for y in range(0,h-size,step):
            small_img = img[y:y+size,x:x+size]
            small_img = small_img.astype("float") / 255.0
            images_to_pred.append(small_img)
            X.append(x)
            Y.append(y)
    
    labels = [None] * len(X)
    
    return (np.array(images_to_pred),np.array(labels),np.array(X),np.array(Y))

In [11]:
# What images should we read in for segmentation
img = cv2.imread("aaron_ROI_test_original/2016_05_19.png")
#img = cv2.imread("20mb_pngs/Morley_180611.png")
(images, labels, x_values, y_values) = get_small_imgs_from_mosaic(img)

print(len(images))

134980


In [1]:
# Run the model on the small images extracted from the original
outputs = model.predict(images,verbose=1)

In [11]:
# Draws the areas classified as soil with >98% confidence on
# the original image as well as a black and white mask and
# saves them to disk
h,w = img.shape[:2]
size = 9
step = 6
out_img = img.copy()
out_img_bw = np.zeros((h,w))

for i in range(len(outputs)):
    if outputs[i][1] >= 0.98:
        x = x_values[i]
        y = y_values[i]
        cv2.rectangle(out_img, (x+1,y+1),(x+size-1,y+size-1),(0,0,255),-1)
        cv2.rectangle(out_img_bw,(x+1,y+1),(x+size-1,y+size-1),255,-1)
        #out_img_bw[y+1:y+size-1][x+1:x+size-1] = 1
        #out_img_bw[x+1:x+size-1][y+1:y+size-1] = 1
        
cv2.imwrite("output.png",out_img)

out_img_bw = out_img_bw * 255
out_img_bw.astype("uint8")
#kernel = np.ones((11,11))
#out_img_bw =cv2.dilate(out_img_bw,kernel,1)
#out_img_bw = cv2.morphologyEx(out_img_bw, cv2.MORPH_CLOSE, kernel)
cv2.imwrite("output_bw.png",out_img_bw)

True

In [130]:
# Combines lines that are too close together, ideally
# resulting in only 1 line between plots.
def line_consensus(lines):
    lines.sort()
    cons = []
    #print(lines)

    for i in range(len(lines)-1):
        if lines[i+1] - lines[i] < 10:
            avg = (lines[i+1] + lines[i]) / 2
            cons.append(avg)
        # Special case for item first in list
        elif lines[i] - lines[i-1] >= 10 and i != 0:
            #continue
            cons.append(lines[i])
        elif i == 0:
            cons.append(lines[i])
    
    # Special case for item last in list
    if lines[-1] - lines[-2] >= 10:
        cons.append(lines[-1])
    
    return cons

In [153]:
# Creates a mask image from the x and y coordinates of
# vertical and horizontal lines, respectively.
def mask_write(shape,ver_cons,hor_cons,name):
    mask = np.zeros(shape)
    
    # Vertical lines
    for line in ver_cons:
        x = int(line)
        y0 = 0
        y1 = shape[0]
        cv2.line(mask,(x,y0),(x,y1),255,1)
    
    for line in hor_cons:
        y = int(line)
        x0 = 0
        x1 = shape[1]
        cv2.line(mask,(x0,y),(x1,y),255,1)
    
    cv2.imwrite(name,mask)
    return mask

In [142]:
# Because the fields have small plots that make it so the
# lines between plots are not uniformly spaced horizontally
# (on these images, it could work the other way too) I
# wanted to see if finding the edge lines and then simply
# equalizing the distance between all the intervening lines
# would work. It didn't. Either the plots are not exactly
# the same width, or pixel width is not uniform across the
# entire image.
def vert_equalize(lines):
    distances = []
    ret_lines = []
    for i in range(len(lines)-1):
        distances.append(lines[i+1] - lines[i])
    
    print(np.mean(distances))
    mean = np.mean(distances)
    least = lines[0]
    most = lines[-1]
    for i in range(len(lines)):
        if i == 0:
            ret_lines.append(least)
        elif i == len(lines)-1:
            ret_lines.append(most)
        else:
            ret_lines.append(least + mean * i)
    
    return ret_lines
        

In [72]:
# Draw all the Hough Lines on the image.
# Save a black and white mask, and the original
# with lines drawn on it.
out = img.copy()
#cv2.imwrite("test.png",out)
### Transition to using Hough lines
#out_img_bw = cv2.imread("output_bw.png")
mask = out_img_bw
hough_bw = np.zeros((h,w))
#edges = cv2.Canny(out_img,50,150,aperture_size=3)
lines = cv2.HoughLines(mask.astype("uint8"),1,np.pi/90,1500)
#print(lines.shape)
for line in lines:
    #print(line)
    for rho,theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 10000*(-b))
        y1 = int(y0 + 10000*(a))
        x2 = int(x0 - 10000*(-b))
        y2 = int(y0 - 10000*(a))
    
        cv2.line(out,(x1,y1),(x2,y2),(0,0,255),5)
        cv2.line(hough_bw,(x1,y1),(x2,y2),255,5)


hough_bw = np.bitwise_not(hough_bw.astype("uint8"))

cv2.imwrite('houghlines.png', out)
cv2.imwrite('houghlines_bw.png', hough_bw)

(170, 1, 2)


True

In [155]:
# Separate the Hough lines into horizontal and vertical lines

### I want to combine the nearby lines rather than just making them thicker.
hor = []
ver = []

for line in lines:
    if line[0][1] < 0.1:
        ver.append(line[0,0])
    else:
        hor.append(line[0,0])

hor.sort()
ver.sort()
ver2 = ver
hor2 = hor
hor_cons = []
ver_cons = []

while len(hor_cons) != len(hor):
    hor_cons = hor
    hor = line_consensus(hor)
    
while len(ver_cons) != len(ver):
    ver_cons = ver
    ver = line_consensus(ver)
    
ver_eq = vert_equalize(ver_cons)
#Making the horizontal distances equal is not as accurate, because the plots are not all perfectly the same size.
#print(len(ver_cons))
#print(len(ver_eq))


# Write out various images using different sets of lines.
mask = mask_write((h,w),ver_cons,hor_cons,"test0.png")
mask = np.bitwise_not(mask.astype("uint8"))
cv2.imwrite("mask2.png",mask)
mask_write((h,w),ver_cons,hor_cons,"test.png")
mask_write((h,w),ver2,hor2,"test2.png")

cutout = img.copy()
output = cv2.bitwise_and(cutout,cutout,mask=mask)
print(cutout.shape)
cv2.imwrite("test3.png",output)

41.44897959183673
(2390, 2045, 3)


True

In [32]:
# Write an image that should only include the areas that are in plots, but will also include small plots.
cutout = img

output = cv2.bitwise_and(cutout,cutout,mask=hough_bw)
print(cutout.shape)
print(hough_bw.shape)
cv2.imwrite("cutout.png",output)

#b,g,r = cv2.split(img)
#b = b & hough_bw
#g = g & hough_bw
#r = r & hough_bw

#cutouts = cv2.merge(b,g)
#cutouts = cv2.merge(cutouts,r)

(2390, 2045, 3)
(2390, 2045)


True

In [163]:
### Extract plots  49x16
# I was using hard-coded values to see about how big the area of each plot is
# but it should be detected automatically, using the numbers of rows
# and columns.
plots = []

im2,contours,hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x,y,width,height = cv2.boundingRect(contour)
    area = width*height
    plots.append((x,y,width,height,area))

areas = []
for plot in plots:
    areas.append(plot[4])

area_set = list(set(areas))
hist = []
#print(area_set)

for i in area_set:
    count = 0
    for area in areas:
        if area == i:
            count += 1
    
    hist.append(count)

#plt.bar(hist,area_set)
#plt.bar(area_set,hist)
#plt.show()

plot2 = [plot for plot in plots if plot[4] > 2100]
print(len(plot2))
#49*16

784


In [164]:
# Create images that are the final mask, as well as the original image
# with all the non-plot regions removed.
final_mask = np.zeros(hough_bw.shape)
for plot in plot2:
    final_mask[plot[1]:plot[1]+plot[3],plot[0]:plot[0]+plot[2]] = 1


plots_img = cv2.bitwise_and(cutout,cutout,mask=final_mask.astype("uint8"))    

cv2.imwrite("plots.png",plots_img)
cv2.imwrite("final_mask.png",final_mask)

True

In [168]:
# Save each plot image in a folder, with the row and column
# information in its filename.
# TODO: This will be changed to saving each plot in a unique
# folder, and then saving images from different days together.
if not os.path.exists("unique_plots"):
    os.mkdir("unique_plots")
    
if plot2[0][0] > plot2[-1][0]:
    plot2 = list(reversed(plot2))

grid_row = 0
grid_col = 0
raw_x = plot2[0][1]
    
for plot in plot2:
    if plot[1] != raw_x:
        grid_row += 1
        grid_col = 0
        raw_x = plot[1]
    name = "unique_plots/" + str(grid_row) + "_" + str(grid_col) + ".png"
    cv2.imwrite(name,plots_img[plot[1]:plot[1]+plot[3],plot[0]:plot[0]+plot[2]])
    grid_col += 1

In [180]:
# Find the weighted centroid of the plot based on greenness,
# which should be the center of the plot.
# We can then extrapolate that information to images later in
# a series.

# NOT FINISHED
def centroid(points):
    total = 0
    weighted_total = 0
    for i in range(len(points)):
        weighted_total += points[i]*(i+1)
        total += points[i]

    mean = total / len(points)
    centroid = weighted_total / len(points) / mean
    return centroid - 1

In [183]:
# Testing code
points = []
print (centroid(points))


1.0
